## Telecom Customer Churn Retention

### Problem Statement

Customer churn can be an important problem for subscription-based companies. Loss of a customer leads to loss of revenue. 

The goal of the project is to identify key factors that lead to the churn outcome and improve customer retention.

### Data

- Source: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
- This dataset has 7043 rows and 21 columns

### Importing Data and Packages

In [56]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Importing the Dataset

In [57]:
df = pd.read_csv(r'data\WA_Fn-UseC_-Telco-Customer-Churn.csv')

### Printing top 10 records

In [58]:

pd.set_option('display.max_columns', None)
df.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


### Dataset Overview

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


### Observations: 

customerID: Unique ID assigned to each customer

gender: Gender of the customer → (Male / Female)

SeniorCitizen: Whether the customer is a senior citizen → (0 = No, 1 = Yes)

Partner: Whether the customer has a partner → (Yes / No)

Dependents: Whether the customer has dependents → (Yes / No)

tenure: Number of months the customer has stayed with the company

PhoneService: Whether the customer has a phone service → (Yes / No)

MultipleLines: Whether the customer has multiple phone lines → (Yes / No / No phone service)

InternetService: Type of internet service → (DSL / Fiber optic / No)

OnlineSecurity: Whether the customer has online security add-on → (Yes / No / No internet service)

OnlineBackup: Whether the customer has online backup add-on → (Yes / No / No internet service)

DeviceProtection: Whether the customer has device protection add-on → (Yes / No / No internet service)

TechSupport: Whether the customer has tech support add-on → (Yes / No / No internet service)

StreamingTV: Whether the customer streams TV → (Yes / No / No internet service)

StreamingMovies: Whether the customer streams movies → (Yes / No / No internet service)

Contract: Type of customer contract → (Month-to-month / One year / Two year)

PaperlessBilling: Whether the customer uses paperless billing → (Yes / No)

PaymentMethod: Payment method used by the customer

MonthlyCharges: Amount charged per month

TotalCharges: Total amount charged to the customer

Churn: Whether the customer left the company → (Yes / No)

### Data Checks

- Missing Values
- Duplicates
- Unusual Data Types
- Unique Values
- Numerical Trends

### Finding any missing values

In [60]:
missing_values = df.isna().sum() # Print to see each column

if missing_values.sum() == 0:
    print('No missing values in the dataset.')
else: 
    for col, count in missing_values.items():
        if count > 0:
            print(f"There are {count} missing values in the '{col}'.")

No missing values in the dataset.


### Finding any duplicate values

In [61]:
duplicate_count = df.duplicated().sum()

if duplicate_count == 0:
    print("No duplicate rows in the dataset.")
else:
    print(f"There are {duplicate_count} duplicate rows in the dataset.")


No duplicate rows in the dataset.


### Finding any incorrectly classified data types in columns

In [62]:
obj_cols = df.select_dtypes(include='object').columns

flagged = []

for col in obj_cols:
    converted = pd.to_numeric(df[col].astype(str).str.strip(), errors='coerce')

    if converted.notna().any() and not converted.notna().all():
        print(f"! '{col}' looks numeric but has some non-numeric values.")
        flagged.append(col)

if not flagged:
    print("No numeric-like object columns found.")


! 'TotalCharges' looks numeric but has some non-numeric values.


Note: Converting TotalCharges to numeric since it contains numeric values, with some blank entries treated as missing values.

In [63]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

### Checking unique values to get a better picture

In [96]:
for col in df.columns:
    print(f"Column name: {col}")
    print(f"Number of unique values: {df[col].nunique()}")
    print("Unique values:", df[col].unique())
    print("-" * 100)

Column name: gender
Number of unique values: 2
Unique values: ['Female' 'Male']
----------------------------------------------------------------------------------------------------
Column name: SeniorCitizen
Number of unique values: 2
Unique values: [0 1]
----------------------------------------------------------------------------------------------------
Column name: Partner
Number of unique values: 2
Unique values: ['Yes' 'No']
----------------------------------------------------------------------------------------------------
Column name: Dependents
Number of unique values: 2
Unique values: ['No' 'Yes']
----------------------------------------------------------------------------------------------------
Column name: tenure
Number of unique values: 73
Unique values: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]

Note: The customerID column does not provide any predictive or descriptive value for the analysis. It will be excluded from the dataset to avoid introducing noise into the modeling process.

In [65]:
df = df.drop('customerID', axis=1)

#### Feature extraction

In [100]:
numerical_features = [f for f in df.columns if df[f].dtype != 'object']
categorical_features = [f for f in df.columns if df[f].dtype == 'object']

print("Numerical Features: ",numerical_features)
print("Categorical Features: ", categorical_features)

Numerical Features:  ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']
Categorical Features:  ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn']


### Analyzing the numerical features

In [101]:
df[numerical_features].corr()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
SeniorCitizen,1.000000,0.016567,0.220173,0.102411
tenure,0.016567,1.000000,0.247900,0.825880
MonthlyCharges,0.220173,0.247900,1.000000,0.651065
TotalCharges,0.102411,0.825880,0.651065,1.000000


In [102]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
SeniorCitizen,7043.0,0.162147,0.368612,0.00,0.00,0.000,0.0000,1.00
tenure,7043.0,32.371149,24.559481,0.00,9.00,29.000,55.0000,72.00
MonthlyCharges,7043.0,64.761692,30.090047,18.25,35.50,70.350,89.8500,118.75
TotalCharges,7032.0,2283.300441,2266.771362,18.80,401.45,1397.475,3794.7375,8684.80


What we know so far: 

* `tenure` and `TotalCharges` show a strong positive correlation (0.82), as total charges grow with customer tenure.
* `MonthlyCharges` has a moderate correlation (0.65) with `TotalCharges`, reflecting higher bills leading to higher totals.
* `SeniorCitizen` is binary with \~16% seniors and shows weak correlation with other numeric features.
* `tenure` ranges from 0–72 months, while `MonthlyCharges` range from \$18.25 to \$118.75, indicating varied service plans.
* `TotalCharges` is highly variable, from \$18.80 to \$8,684.80, driven by differences in tenure and plan pricing.

This shows that there is a need for further analysis combined with all the numerical and categorical features. 